In [16]:
import os
import pickle
from contextlib import nullcontext
import torch
import torch.nn as nn
from model import GPTConfig, GPT

## Inicializar modelo pre-entrenado

Setup del modelo:

In [3]:
out_dir = 'out'
start = ''
num_samples = 10
max_new_tokens = 500
temperature = 0.9
#top_k = 200
seed = 33313988
device = 'cuda'
dtype='float16'

In [4]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

Cargar configuraciones del checkpoint e inicializarlo.

In [5]:
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

<All keys matched successfully>

Esta es la configuración del último checkpoint:

In [6]:
gptconf

GPTConfig(block_size=128, vocab_size=526, n_layer=3, n_head=6, n_embd=384, dropout=0.0, bias=False)

Arquitectura del modelo:

In [7]:
model

GPT(
  (token_embedding_table): Embedding(526, 384)
  (position_embedding_table): Embedding(128, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
     

Número de parámetros:

In [10]:
print(f"Número de parámetros GPT-sentiment checkpoint: {model.get_num_params()/1e6:.2f} millones")

Número de parámetros GPT-sentiment checkpoint: 5.52 millones


Cargar tokenizador y crear funciones `encode` y `decode`:

In [11]:
model.eval()
model.to(device)

with open('./data/by_char/meta.pkl', 'rb') as f:
    meta = pickle.load(f)
    vocab_size = meta['vocab_size']
    itos = meta['itos']
    stoi = meta['stoi']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])


## Generación de muestras

Crear una función para generar muestras y concatenarlas.

Esta función se utiliza en `train.py` para capturar samples en 
cada loop de evaluación del modelo. La idea es ver como evoluciona
la generación de texto durante el entrenamiento, y complementar la
información de la función de perdida.

In [12]:
@torch.no_grad()
def get_samples(num_samples, max_new_tokens=10):
    model.eval()
    out = []
    for k in range(num_samples):
            y = model.generate(idx=torch.zeros((1, 128), dtype=torch.long, device=device), max_new_tokens=max_new_tokens, temperature=temperature)
            out.append(f"({k+1}) {decode(y[0][128:].tolist())}")
    model.train()
    return '\n'.join(out)

In [13]:
print(get_samples(20, max_new_tokens=250))

(1)                     asueltos ?  ondiarte no si no, solo hay carter leí, un hombre 1 famos inflar del descuento y arriesinvierno no se debe ser tratadas por decir de lo que lo dijo esa huello es de #senansventonces https://t.co/MXpdjlxcWN"@user Por co
(2)   @user         Alguien!!!!     @user ...@user Si super partimado ctm el igual que ellos arráño en trans malos en la locurra de segundar alfartos y extranjeros todos los días indios y se me inefasto recrecto con puesto de mierda.Esta gente más corrup
(3)                  htmerosMe hace me sosto a votar cuando... q como respreció a los chilenos..en la misma votomada... Ahora estás el potito no sexual de la oportunidad con negra a esta sociedad...contribuyen de programas tienen 🤣 https://t.co/A ZWwlqlb
(4)   😂NT 👩‍[6/22221    🔹✔️🤣 https://t.co/xP6fpZC469@user Segunda   Me lastre ctm, hasta homosexual, como se puede ir incluso de jóvenes según putas de verdaderos a madre y muchos ! 🙌🔵[Listos sigan iguales se va a la semana de toda la 

## GPTClassifier

**Objetivo:** user modelo GPT-sentiment pre-entrenado y agregar una
cabeza de clasificación para adaptar la representación del texto
a discriminar a que clase pertenece cada uno.

In [17]:
class GPTClassifier(nn.Module):
    """
    This model is a simple MLP with one embedding layer with the weights of the Llama model.
    """
    def __init__(self, gpt_backbone, n_classes=3, freeze=True):
        super(GPTClassifier, self).__init__()
        self.gpt_backbone = gpt_backbone

        # freeze parameters of the gpt backbone
        if freeze:
            for param in self.gpt_backbone.parameters():
                param.requires_grad = False

        # add new classification head
        self.gpt_backbone.lm_head = nn.Linear(gpt_backbone.lm_head.in_features, n_classes)
        
    def forward(self, x):
        return self.gpt_backbone(x)

In [22]:
clf = GPTClassifier(model, n_classes=3, freeze=True)
print(f"Nueva cábeza del modelo ---> {clf.gpt_backbone.lm_head}")

Nueva cábeza del modelo ---> Linear(in_features=384, out_features=3, bias=True)


Preparar los datos y verificar que fluyan correctamente por el modelo.